# Principal Component Analysis (PCA)

Principal Component Analysis (PCA) is a dimensionality reduction technique that is widely used in data analysis and machine learning. It transforms the data into a new coordinate system such that the greatest variance by any projection of the data comes to lie on the first coordinate (called the first principal component), the second greatest variance on the second coordinate, and so on. This technique helps in reducing the number of variables while preserving as much information as possible.

## SVD

SVD is a subset of PCA. It is a matrix factorization technique that factors a matrix into three matrices: U, Σ, and V. The SVD of a matrix A is given by:

$$A = U \Sigma V^T$$

where:
- U is an m x m orthogonal matrix.
- Σ is an m x n diagonal matrix with non-negative real numbers on the diagonal.

$A A^T$ and $A^T A$ are symmetric matrices, so they have an eigendecomposition:

$$A A^T = U \Sigma \Sigma^T U^T$$

$A^T A$ is a symmetric matrix, so it has an eigendecomposition:

$$A^T A = V \Sigma^T \Sigma V^T$$

In [1]:
import numpy as np

# SVD
X = np.array([[4, 1, 3], [7, 5, 9], [2, 4, 9], [1, 2, 3]])
XXT = np.dot(X, X.T)
XTX = np.dot(X.T, X)
ei_values, ei_vectors = np.linalg.eig(XXT)
ei_values2, ei_vectors2 = np.linalg.eig(XTX)
S = np.diag(np.sqrt(ei_values))
S = S[: XXT.shape[0], : XTX.shape[0]]
U = ei_vectors
V = ei_vectors2
print(U)
print(S)
print(V)
print(np.dot(np.dot(U, S), V.T))

[[ 0.27521645  0.54830912  0.62935278  0.47699905]
 [ 0.73735103  0.40455249 -0.39371346 -0.37099926]
 [ 0.57730663 -0.70752362  0.4041267  -0.05299989]
 [ 0.21745439 -0.18736255 -0.53440284  0.79499841]]
[[16.74544917  0.          0.        ]
 [ 0.          3.84952987  0.        ]
 [ 0.          0.          0.87809551]
 [ 0.          0.          0.        ]]
[[ 0.45590838  0.88912003  0.04016359]
 [ 0.4004734  -0.16462997 -0.90139782]
 [ 0.79483873 -0.42703927  0.43112511]]
[[4. 1. 3.]
 [7. 5. 9.]
 [2. 4. 9.]
 [1. 2. 3.]]


# Non-negative Matrix Factorization (NMF)

Non-negative Matrix Factorization (NMF) is a dimensionality reduction technique that is widely used in data analysis and machine learning. It factorizes the given non-negative matrix into two non-negative matrices such that the product of these two matrices closely approximates the original matrix. This technique helps in reducing the number of variables while preserving as much information as possible.

*The main advantage of NMF* compared to previously introduced low-rank models is that the nonnegativity constraints on the factors W and H lead to an easily interpretable part-based decomposition. In application like image processing and text mining.


$$ X \approx W H $$

where:
- $X$ is an m x n non-negative matrix.
- $W$ is an m x r non-negative matrix.
- $H$ is an r x n non-negative matrix.

# Simplex Structure Matrix Factorization (SSMF)

SSMF is a generalization of NMF.

$$ X \approx W H $$

where:
- $X$ is an m x n non-negative matrix.
- $W$ is an m x r non-negative matrix.
- $H$ is column-stochastic matrix.
$$
H(:,j) \in \Delta^{r} for\ all\ j,\\\Delta^{r} = \{x \in R^{r} | x \geq 0, e^T x = 1\}\\e = \begin{bmatrix}1\\1\\\vdots\\1\end{bmatrix}
$$

*The main advantage of SSMF* is that the column-stochastic constraints on the factors H lead to an easier interpretation of the factors. The factors in H can be interpreted as probability distributions over the columns of X.

## why SSMF is a generalization of NMF?

1. Normalize the input matrix $X$ s.t. $X \in \Delta^{m \times n}$
$$ X^Te = e $$
2. Normalize the factor matrix $W$ s.t. $W \in \Delta^{m \times r}$
$$ W^Te = e $$

Then, we have:
$$ X \approx W H $$
$$ X^Te = (W H)^Te = H^TW^Te = H^Te = e $$
$$ H^Te = e $$
$$ H(:,j) \in \Delta^{r} for\ all\ j $$

$$ \frac{X}{\|X\|_1} = \frac{W}{\|W\|_1} \frac{H \cdot {\|W\|_1}^T}{\|X\|_1} $$

In [5]:
X = np.array([[4, 1, 3], [7, 5, 9], [2, 4, 9], [1, 2, 6]])
# nmf
from sklearn.decomposition import NMF
nmf = NMF(n_components=2)
W = nmf.fit_transform(X)
H = nmf.components_
print(W)
print(H)
np.dot(W, H)

[[0.71786685 0.13548403]
 [1.15088496 0.98347962]
 [0.11221287 1.4110256 ]
 [0.         0.93836125]]
[[5.26575428 1.77185679 2.65873193]
 [1.00610417 2.62251602 6.19591016]]


array([[3.91642151, 1.62726631, 2.74806243],
       [7.04976036, 4.6183944 , 9.15344599],
       [2.01052413, 3.89926239, 9.04093182],
       [0.94408916, 2.4608674 , 5.81400198]])

In [27]:
# SSMF one way
normX1 = np.sum(X, axis=0)
normW1 = np.sum(W, axis=0)
X_norm = X / normX1
W_norm = W / normW1
H_new = ((H / normX1).T * normW1).T
print(W_norm)
print(H_new)
print(X_norm)

print(np.dot(H_new.T, np.ones((2,1))))

[[0.36238246 0.03906296]
 [0.58097197 0.28355831]
 [0.05664557 0.40682901]
 [0.         0.27054972]]
[[0.74509095 0.29249881 0.19506867]
 [0.24925157 0.75798373 0.79591067]]
[[0.28571429 0.08333333 0.11111111]
 [0.5        0.41666667 0.33333333]
 [0.14285714 0.33333333 0.33333333]
 [0.07142857 0.16666667 0.22222222]]
[[0.99434251]
 [1.05048254]
 [0.99097934]]


# Essential Uniqueness

## Definition
Let $X \in \Re^{m \times n}$, and $r \leq min(m, n)$ be an integer. Let $(W,H)$ be a solution of $X = WH$ if and only if any other pair $(W',H') \in \Re^{m \times r}\times \Re^{r\times n}$ that satisfies $X = W'H'$ for all k, $$W'(:,k)=\alpha _kW(:,\pi (k))$$ and $$H'(k,:)=\alpha _k^{-1}H(\pi (k),:)$$ where $\pi$ is a permutation of $\{1,2,...,r\}$ and $\alpha _k \neq 0$ for all k.